In [16]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
from nltk.corpus import stopwords
import nltk

import pickle
import spacy

In [17]:
df = pd.read_csv('TASS2019_country_UY_train-sintilde.csv')
df.head()

,Unnamed: 0,tweetid,user,content,date,lang,sentiment
0,0,768529421548609536,293627746,yo espero tranquila si gana de nvo alg partido...,2016-08-24 19:24:27+00:00,es,neu
1,1,771028255629512704,35709180,"los fracasos de michelle, cristina y dilma mue...",2016-08-31 16:53:55+00:00,es,0
2,2,771055632854183936,146577758,debemos orar por los hnos venezolanos gente q ...,2016-08-31 18:42:43+00:00,es,0
3,3,770138191689093120,355883744,me dijo que no le habia gustado no se cuantas ...,2016-08-29 05:57:08+00:00,es,neu
4,4,769716104491040768,2701106364,jajaja yo fui el viernes precavido pero el f...,2016-08-28 01:59:54+00:00,es,0


In [18]:
df['sentiment'].value_counts()

0      367
1      290
neu    192
Name: sentiment, dtype: int64

In [19]:
df['sentiment'] = df['sentiment'].astype('category')
df['sentiment'] = df['sentiment'].cat.codes

In [20]:
df['sentiment'].value_counts()

0    367
1    290
2    192
Name: sentiment, dtype: int64

In [21]:
#df_1 = df[df['sentiment'] == 1]
#df_0 = df[df['sentiment'] == 0].sample(int(len(df.loc[df['sentiment'] == 0]) * 0.7150))

In [22]:
#df_super = df_1.append(df_0)

In [23]:
#df_super['sentiment'].value_counts()

In [24]:
#df = df_super

In [25]:
df.iloc[0]['content']

'yo espero tranquila si gana de nvo alg partido tradicional, despues se q podre pasarla mal, pero tamb me voy a reir bastante'

In [26]:
df.shape

(849, 7)

In [27]:
nlp = spacy.load("es_core_news_sm")
stops = stopwords.words("spanish")

In [32]:
def normalize(comment):
     comment = nlp(comment.lower())
     lemmatized = list()
     for word in comment:
          word.lemma_.strip()
          if (word.pos_ == 'NOUN' or word.pos_ == 'VERB' or word.pos_ == 'ADJ') and word.is_alpha:
               lemma = word.lemma_  
               if lemma and lemma not in stops:     
                    lemmatized.append(lemma)
     return " ".join(lemmatized)

In [33]:
X = df['content'].apply(normalize)
y = df['sentiment']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [35]:
count_vectorizer = CountVectorizer(ngram_range=(1,3), min_df=2)
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

In [36]:
bow_df_train = pd.DataFrame(X_train_count.A, columns=count_vectorizer.get_feature_names())
bow_df_test = pd.DataFrame(X_test_count.A, columns=count_vectorizer.get_feature_names())

In [37]:
bow_df_train.head()

,abrazar,abrazar viejo,abrazarla,aburrir,aca,acabar,acordar,acostumbrar,adorable,adorar,...,viajar,vida,viejo,viernes,vivir,volver,votar,xq,zapato,zorro
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
nb_classifier = MultinomialNB()
nb_classifier.fit(bow_df_train, y_train)
pred = nb_classifier.predict(bow_df_test)

# accuracy (veces que fue bien, de las veces que me dio +) 
# 
# recall
# f1 

score = metrics.f1_score(y_test, pred, average='macro')
score

0.4409700578015861

In [39]:
cm = metrics.confusion_matrix(y_test, pred)
cm

array([[88, 21, 20],
       [33, 53,  6],
       [37, 17,  6]], dtype=int64)

In [40]:
weights = sorted(zip(nb_classifier.coef_[0], count_vectorizer.get_feature_names()), reverse=True)
weights[:20]

[(-3.604138225658846, 'ser'),
 (-3.8554526539397522, 'tener'),
 (-4.243218184948516, 'ir'),
 (-4.35444382005874, 'hacer'),
 (-4.4796069630127455, 'decir'),
 (-4.62270780665342, 'querer'),
 (-4.702750514326956, 'ver'),
 (-4.789761891316585, 'poder'),
 (-4.88507207112091, 'pensar'),
 (-4.88507207112091, 'bueno'),
 (-4.990432586778736, 'sentir'),
 (-4.990432586778736, 'coser'),
 (-5.10821562243512, 'tremendo'),
 (-5.10821562243512, 'solo'),
 (-5.10821562243512, 'gente'),
 (-5.10821562243512, 'dormir'),
 (-5.10821562243512, 'dia'),
 (-5.241747015059643, 'tomar'),
 (-5.241747015059643, 'nuevo'),
 (-5.241747015059643, 'malo')]

In [42]:
filename = 'modelo_transformador_proyecto.sav'

with open(filename, 'wb') as file:
    pickle.dump((count_vectorizer, nb_classifier), file)

In [59]:
x = pd.DataFrame(['6565 656565 65656 656'])
x = count_vectorizer.transform(x.iloc[0])

print(nb_classifier.predict_proba(x))
print(nb_classifier.predict(x))

[[0.41901408 0.34859155 0.23239437]]
[0]
